In [1]:
import numpy as np
import cv2 as cv
import face_recognition
import glob
from PIL import Image
import sys
from time import sleep
from pylibfreenect2 import *

from matplotlib import pyplot as plt
from ipywidgets import *
import pandas as pd
from pyntcloud import PyntCloud

In [5]:
# parameters
LEFT_CAMERA = "015001743347".encode('utf-8')
RIGHT_CAMERA = "009747143547".encode('utf-8')

RGB_IMG_COUNTER = 100
IR_IMG_COUNTER = 0
D_IMG_COUNTER = 0
R_RGB_IR_IMG_COUNTER = 0
L_RGB_IR_IMG_COUNTER = 0
LIVE_STREAM_OPTIONS = {"rgb_l": 1, "ir_l":0, "d_l":0, "rgb_r": 1, "ir_r":0, "d_r":0}
LIVE_STREAM_CHOICE = 0
RECORD = 0
RECORD_OPTIONS = {"rgb_l": False, "ir_l":False, "d_l":False, "rgb_r": False, "ir_r":False, "d_r":False}
RECORD_PATH = {"rgb_l": 'videoRGB1.mp4', "ir_l":'videoIR1.mp4', "d_l":'videoDepth1.mp4', "rgb_r": 'videoRGB2.mp4', "ir_r":'videoIR2.mp4', "d_r":'videoDepth2.mp4'}
SCREENSHOT_PATH = {"rgb_l": 'photos/my_calib/stereo/calib_img_rgb_l', "ir_l":'photos/my_calib/stereo/calib_img_ir_l', "d_l":'photos/my_calib/stereo/calib_img_d_l', "rgb_r": 'photos/my_calib/stereo/calib_img_rgb_r', "ir_r":'photos/my_calib/stereo/calib_img_ir_r', "d_r": 'photos/my_calib/stereo/calib_img_d_r'}

face_recog_scale_factor = 4
previousTimestamp = cv.getTickCount()

stereo_map = cv.FileStorage("improved_params10.xml", cv.FILE_STORAGE_READ)
Left_Stereo_Map_x = stereo_map.getNode("Left_Stereo_Map_x").mat()
Left_Stereo_Map_y = stereo_map.getNode("Left_Stereo_Map_y").mat()
Right_Stereo_Map_x = stereo_map.getNode("Right_Stereo_Map_x").mat()
Right_Stereo_Map_y = stereo_map.getNode("Right_Stereo_Map_y").mat()
stereo_map.release()

inv_stereo_map = cv.FileStorage("Inv_stereo_map.xml", cv.FILE_STORAGE_READ)
Inv_Left_Stereo_Map_x = inv_stereo_map.getNode("Inv_Left_Stereo_Map_x").mat()
Inv_Left_Stereo_Map_y = inv_stereo_map.getNode("Inv_Left_Stereo_Map_y").mat()
Inv_Right_Stereo_Map_x = inv_stereo_map.getNode("Inv_Right_Stereo_Map_x").mat()
Inv_Right_Stereo_Map_y = inv_stereo_map.getNode("Inv_Right_Stereo_Map_y").mat()
inv_stereo_map.release()


def calculateCenterPoint(pointArr):
    accumulater = [0,0]
    for point in pointArr:
        accumulater[0] += point[0]
        accumulater[1] += point[1]
    length = len(pointArr)
    return [accumulater[0] / length, accumulater[1] / length]

def processFeatureLocation(face_landmarks_list):
    eyeLocations = {}
    for face_landmarks in face_landmarks_list:
        for facial_feature in face_landmarks.keys():
            if facial_feature == 'left_eye' or facial_feature == 'right_eye':
            #print("The {} in this face has the following points: {}".format(facial_feature, face_landmarks[facial_feature]))
                feature_points = np.array(face_landmarks[facial_feature])
                if len(feature_points) > 0:
                    with np.nditer(feature_points, op_flags=['readwrite']) as it:
                        for x in it:
                            x*=face_recog_scale_factor
                    eyeLocations[facial_feature] = feature_points
    return eyeLocations


def getEyeLocations(np_color):
    small_frame = cv.resize(np_color, (0, 0), fx=1 / face_recog_scale_factor, fy=1 / face_recog_scale_factor)
    rgb_frame = small_frame[:, :, ::-1]
    face_landmarks_list = face_recognition.face_landmarks(rgb_frame, model='small')
    return processFeatureLocation(face_landmarks_list)
    
def markEye(np_color):
    eyeLocations = getEyeLocations(np_color)
    frame = np_color
    for eyeKey in eyeLocations:
        center = calculateCenterPoint(eyeLocations[eyeKey])
        frame = cv.drawMarker(frame.copy(), (int(center[0]), int(center[1])), (0, 255, 0), thickness=1)
    return frame


def calculateEyeRemappedLocation(eyeCenter, mapx, mapy):
    x = int(eyeCenter[0])
    y = int(eyeCenter[1])
    if x < 0 or x >= 1920 or y < 0 or y >= 1080:
        return None
    new_x = mapy[y,x]
    new_y = mapx[y,x]
    if new_x < 0 or new_y < 0:
        return None
    return [new_y,new_x]

def markEyeAndRemappedEye(np_color, mapx, mapy):
    eyeLocations = getEyeLocations(np_color)
    frame = np_color
    remappedEyes = {}
    for eyeKey in eyeLocations:
        center = calculateCenterPoint(eyeLocations[eyeKey])
        remappedCenter = calculateEyeRemappedLocation(center, mapx, mapy)
        frame = cv.drawMarker(frame.copy(), (int(center[0]), int(center[1])), (0, 255, 0), thickness=1)
        if remappedCenter:
            remappedEyes[eyeKey] = remappedCenter
            frame = cv.drawMarker(frame.copy(), (int(remappedCenter[0]), int(remappedCenter[1])), (0, 255, 0), thickness=1)
    return (frame, remappedEyes, eyeLocations)
    


def calculateAndShowFPS(frame):
    global previousTimestamp
    tick = cv.getTickCount()
    t = (tick - previousTimestamp)/cv.getTickFrequency()
    previousTimestamp = tick
    storedFPS = 1 / t
    cv.putText(frame, "FPS:" + str("{:.2f}".format(storedFPS * 3)), (0,30), cv.FONT_HERSHEY_SIMPLEX, 1.0, (255,255,255), 1)






In [38]:
#𝚍𝚜𝚝(x,y)=𝚜𝚛𝚌(mapx(x,y),mapy(x,y))

def inverseMapping(mapx, mapy):
    invx = np.empty((1080, 1920), dtype=int); invx.fill(-1)
    invy = np.empty((1080, 1920), dtype=int); invy.fill(-1)
    for row in range(0, len(mapx)):
        for col in range(0, len(mapx[0])):
            inv_row = int(mapy[row, col])
            inv_col = int(mapx[row, col])
            if inv_row < 0 or inv_row >= 1080 or inv_col < 0 or inv_col >= 1920:
                continue

            invx[inv_row, inv_col] = col
            invy[inv_row, inv_col] = row
    return (invx, invy)
        
(Inv_Left_Stereo_Map_x, Inv_Left_Stereo_Map_y) = inverseMapping(Left_Stereo_Map_x, Left_Stereo_Map_y)
(Inv_Right_Stereo_Map_x, Inv_Right_Stereo_Map_y) = inverseMapping(Right_Stereo_Map_x, Right_Stereo_Map_y)

cv_file = cv.FileStorage("Inv_stereo_map.xml", cv.FILE_STORAGE_WRITE)

cv_file.write("Inv_Left_Stereo_Map_x",Inv_Left_Stereo_Map_x)
cv_file.write("Inv_Left_Stereo_Map_y",Inv_Left_Stereo_Map_y)
cv_file.write("Inv_Right_Stereo_Map_x",Inv_Right_Stereo_Map_x)
cv_file.write("Inv_Right_Stereo_Map_y",Inv_Right_Stereo_Map_y)
cv_file.release()

[[  -1   -1   -1 ... 1893 1895 1896]
 [  -1   -1   -1 ... 1893 1894 1896]
 [  -1   -1   -1 ... 1893 1894 1895]
 ...
 [  19   20   21 ... 1880 1881 1882]
 [  19   20   21 ... 1880 1881 1882]
 [  19   20   21 ... 1880 1881 1882]]
[[  -1   -1   -1 ...    3    3    3]
 [  -1   -1   -1 ...    4    4    4]
 [  -1   -1   -1 ...    5    5    5]
 ...
 [1035 1035 1035 ... 1061 1061 1061]
 [1036 1036 1036 ... 1062 1062 1062]
 [1037 1037 1037 ... 1063 1063 1063]]
[[  45   46   47 ... 1877 1878 1879]
 [  45   46   47 ... 1877 1878 1879]
 [  45   46   47 ... 1877 1878 1879]
 ...
 [  41   42   43 ... 1890 1891 1892]
 [  41   42   43 ... 1890 1891 1893]
 [  41   42   43 ... 1890 1892 1893]]
[[  44   44   44 ...   29   29   29]
 [  45   45   45 ...   30   30   30]
 [  46   46   46 ...   31   31   31]
 ...
 [1071 1071 1071 ... 1071 1071 1071]
 [1072 1072 1072 ... 1072 1072 1072]
 [1073 1073 1073 ... 1073 1073 1073]]


In [32]:
pt = (500, 300)
(Inv_Left_Stereo_Map_x[pt[0], pt[1]], Inv_Left_Stereo_Map_y[pt[0], pt[1]])
#Inv_Left_Stereo_Map_x[pt[0], pt[1]]

(316, 481)

In [69]:
pt = (10, 1536)
(Left_Stereo_Map_x[pt[0], pt[1]], Left_Stereo_Map_y[pt[0], pt[1]])

(1561.0164, 4.8464837)

In [34]:
Inv_Left_Stereo_Map_x

array([[  -1,   -1,   -1, ..., 1893, 1895, 1896],
       [  -1,   -1,   -1, ..., 1893, 1894, 1896],
       [  -1,   -1,   -1, ..., 1893, 1894, 1895],
       ...,
       [  19,   20,   21, ..., 1880, 1881, 1882],
       [  19,   20,   21, ..., 1880, 1881, 1882],
       [  19,   20,   21, ..., 1880, 1881, 1882]])

In [7]:
# stereo utils 
def calculateEyeRealWorldLocation(leftEyes, rightEyes, baselineL, focalLength):
    leftEyeDepth = rightEyeDepth = -1
    if 'left_eye' in leftEyes and 'left_eye' in rightEyes:
        ll = leftEyes['left_eye'][0]
        rl = rightEyes['left_eye'][0]
        leftEyeDepth = (baselineL * focalLength) / (ll - rl) - 10
    if 'right_eye' in leftEyes and 'right_eye' in rightEyes:
        lr = leftEyes['right_eye'][0]
        rr = rightEyes['right_eye'][0]
        rightEyeDepth = (baselineL * focalLength) / (lr - rr) - 10
    
    print({'left_eye': leftEyeDepth, 'right_eye': rightEyeDepth})
    return (leftEyeDepth, rightEyeDepth)

from pylibfreenect2 import OpenGLPacketPipeline
pipeline_r = OpenGLPacketPipeline()
pipeline_l = OpenGLPacketPipeline()

print("Packet pipeline:", type(pipeline_r).__name__)

# Create and set logger
logger = createConsoleLogger(LoggerLevel.Debug)
setGlobalLogger(logger)

# set up kinect device
fn = Freenect2()

use_l = use_r = 0
if LIVE_STREAM_OPTIONS["rgb_l"] or LIVE_STREAM_OPTIONS["ir_l"] or LIVE_STREAM_OPTIONS["d_l"]:
    use_l = 1
if LIVE_STREAM_OPTIONS["rgb_r"] or LIVE_STREAM_OPTIONS["ir_r"] or LIVE_STREAM_OPTIONS["d_r"]:
    use_r = 1

device_r = device_l = listener_r = listener_l = 0
if use_r:
    device_r = fn.openDevice(RIGHT_CAMERA, pipeline=pipeline_r)
    if LIVE_STREAM_CHOICE == 0:
        listener_r = SyncMultiFrameListener(FrameType.Color)
    elif LIVE_STREAM_CHOICE == 1:
        listener_r = SyncMultiFrameListener(FrameType.Color | FrameType.Ir | FrameType.Depth)
if use_l:
    device_l = fn.openDevice(LEFT_CAMERA, pipeline=pipeline_l)
    if LIVE_STREAM_CHOICE == 0:
        listener_l = SyncMultiFrameListener(FrameType.Color)
    elif LIVE_STREAM_CHOICE == 1:
        listener_l = SyncMultiFrameListener(FrameType.Color | FrameType.Ir | FrameType.Depth)


if LIVE_STREAM_OPTIONS["rgb_r"]:
    device_r.setColorFrameListener(listener_r)
if LIVE_STREAM_OPTIONS["rgb_l"]:
    device_l.setColorFrameListener(listener_l)
    

if LIVE_STREAM_OPTIONS["ir_r"] or LIVE_STREAM_OPTIONS["d_r"]:
    device_r.setIrAndDepthFrameListener(listener_r)
if LIVE_STREAM_OPTIONS["ir_l"] or LIVE_STREAM_OPTIONS["d_l"]:
    device_l.setIrAndDepthFrameListener(listener_l)
    
if use_r:
    device_r.start()
if use_l:
    device_l.start()

# set video container
fourcc = cv.VideoWriter_fourcc(*'FMP4')
videoRGB_l = videoIR_l = videoDepth_l = videoRGB_r = videoIR_r = videoDepth_r =0
if RECORD_OPTIONS["rgb_l"]:
    videoRGB_l = cv.VideoWriter(RECORD_PATH["rgb_l"], fourcc, 12, (1920, 1080))
if RECORD_OPTIONS["ir_l"]:
    videoIR_l = cv.VideoWriter(RECORD_PATH["ir_l"], fourcc, 12, (512, 424), False)
if RECORD_OPTIONS["d_l"]:
    videoDepth_l = cv.VideoWriter(RECORD_PATH["d_l"], fourcc, 12, (512, 424), False)
if RECORD_OPTIONS["rgb_r"]:
    videoRGB_r = cv.VideoWriter(RECORD_PATH["rgb_r"], fourcc, 12, (1920, 1080))
if RECORD_OPTIONS["ir_r"]:
    videoIR_r = cv.VideoWriter(RECORD_PATH["ir_r"], fourcc, 12, (512, 424), False)
if RECORD_OPTIONS["d_r"]:
    videoDepth_r = cv.VideoWriter(RECORD_PATH["d_r"], fourcc, 12, (512, 424), False)

# debug variables
previousTimestamp = 0
fpsArr = []

while True:
    np_color_r = np_color_l = np_ir_r = np_ir_l = np_depth_r = np_depth_l = 0
    remappedEyes_l = remappedEyes_r = 0
    
    if use_l:
        frames_l = listener_l.waitForNewFrame()
        if LIVE_STREAM_OPTIONS["rgb_l"]:
            color_l = frames_l["color"]
            np_color_l = color_l.asarray()
            np_color_l = np_color_l[:,:,:3]
            markedFrame, remappedEyes_l, _ = markEyeAndRemappedEye(np_color_l, Inv_Left_Stereo_Map_x, Inv_Left_Stereo_Map_y)
            cv.imshow("color_l", markedFrame)

        if LIVE_STREAM_OPTIONS["ir_l"]:
            ir_l = frames_l["ir"]
            np_ir_l = ir_l.asarray()
            np_ir_l /= np_ir_l.max()
            np_ir_l = np.sqrt(np_ir_l)
            #np_ir_l = np.array(Image.fromarray(256 * np_ir_l).convert('L'))
            cv.imshow("ir_l", np_ir_l)

        if LIVE_STREAM_OPTIONS["d_l"]:
            depth_l = frames_l["depth"]
            np_depth_l = depth_l.asarray()
            np_depth_l /= np_depth_l.max()
            np_depth_l = np.sqrt(np_depth_l)
            #np_depth_l = np.array(Image.fromarray(256 * np_depth_l).convert('L'))
            cv.imshow("d_l", np_depth_l)

        listener_l.release(frames_l)
    
    if use_r:
        frames_r = listener_r.waitForNewFrame()
        if LIVE_STREAM_OPTIONS["rgb_r"]:
            color_r = frames_r["color"]
            np_color_r = color_r.asarray()
            np_color_r = np_color_r[:,:,:3]
            markedFrame, remappedEyes_r, _ = markEyeAndRemappedEye(np_color_r, Inv_Right_Stereo_Map_x, Inv_Right_Stereo_Map_y)
            cv.imshow("color_r", markedFrame)

        if LIVE_STREAM_OPTIONS["ir_r"]:
            ir_r = frames_r["ir"]
            np_ir_r = ir_r.asarray()
            np_ir_r /= np_ir_r.max()
            np_ir_r = np.sqrt(np_ir_r)
            #np_ir_r = np.array(Image.fromarray(256 * np_ir_r).convert('L'))
            cv.imshow("ir_r", np_ir_r)

        if LIVE_STREAM_OPTIONS["d_r"]:
            depth_r = frames_r["depth"]
            np_depth_r = depth_r.asarray()
            np_depth_r /= np_depth_r.max()
            np_depth_r = np.sqrt(np_depth_r)
            #np_depth_r = np.array(Image.fromarray(256 * np_depth_r).convert('L'))
            cv.imshow("d_r", np_depth_r)

        listener_r.release(frames_r)
        
    calculateEyeRealWorldLocation(remappedEyes_l, remappedEyes_r, 25, 1060)
    
    if RECORD == True:
        #videoIR1.write(np_ir)
        videoRGB1.write(np_color1)
        #videoDepth1.write(np_depth)
        #videoIR2.write(np_ir)
        videoRGB2.write(np_color2)
        #videoDepth2.write(np_depth)
    
    
    key = cv.waitKey(20)
    if key == ord('r'):
        cv.imwrite(f'{SCREENSHOT_PATH["rgb_r"]}{RGB_IMG_COUNTER}.jpg', np_color_r)
        cv.imwrite(f'{SCREENSHOT_PATH["rgb_l"]}{RGB_IMG_COUNTER}.jpg', np_color_l)
        RGB_IMG_COUNTER+=1
    if key == ord('i'):
        cv.imwrite(f'{SCREENSHOT_PATH["ir_r"]}{IR_IMG_COUNTER}.jpg', np_ir_r)
        cv.imwrite(f'{SCREENSHOT_PATH["ir_l"]}{IR_IMG_COUNTER}.jpg', np_ir_l)
        IR_IMG_COUNTER+=1
    if key == ord('d'):
        cv.imwrite(f'{SCREENSHOT_PATH["d_r"]}{D_IMG_COUNTER}.jpg', np_depth_r)
        cv.imwrite(f'{SCREENSHOT_PATH["d_l"]}{D_IMG_COUNTER}.jpg', np_depth_l)
        D_IMG_COUNTER+=1
    if key == ord('a'):
        cv.imwrite(f'{SCREENSHOT_PATH["ir_r"]}{R_RGB_IR_IMG_COUNTER}.jpg', np_ir_r)
        cv.imwrite(f'{SCREENSHOT_PATH["rgb_r"]}{R_RGB_IR_IMG_COUNTER}.jpg', np_color_r)
        cv.imwrite(f'{SCREENSHOT_PATH["d_r"]}{R_RGB_IR_IMG_COUNTER}.jpg', np_depth_r)
        R_RGB_IR_IMG_COUNTER+=1
    if key == ord('b'):
        cv.imwrite(f'{SCREENSHOT_PATH["ir_l"]}{L_RGB_IR_IMG_COUNTER}.jpg', np_ir_l)
        cv.imwrite(f'{SCREENSHOT_PATH["rgb_l"]}{L_RGB_IR_IMG_COUNTER}.jpg', np_color_l)
        cv.imwrite(f'{SCREENSHOT_PATH["d_l"]}{L_RGB_IR_IMG_COUNTER}.jpg', np_depth_l)
        L_RGB_IR_IMG_COUNTER+=1
    elif key == ord('q'):
        break

if use_l:
    device_l.stop()
    device_l.close()
if use_r:
    device_r.stop()
    device_r.close()

cv.destroyAllWindows()
if RECORD_OPTIONS["rgb_l"]:
    videoRGB_l.release()
if RECORD_OPTIONS["ir_l"]:
    videoIR_l.release()
if RECORD_OPTIONS["d_l"]:
    videoDepth_l.release()
if RECORD_OPTIONS["rgb_r"]:
    videoRGB_r.release()
if RECORD_OPTIONS["ir_r"]:
    videoIR_r.release()
if RECORD_OPTIONS["d_r"]:
    videoDepth_r.release()

    

Packet pipeline: OpenGLPacketPipeline
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye

{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -1, 'right_eye': -1}
{'left_eye': -61.20092378752887, 'right_eye': -59.550561797752806}
{'left_eye': -68.29896907216495, 'right_eye': -64.47688564476886}
{'left_eye': -72.40437158469945, 'right_eye': -1}
{'left_eye': -68.12339331619538, 'right_eye': -65.5940594059406}
{'left_eye': -70.1058201058201, 'right_eye': -66.41604010025063}
{'left_eye': -72.6027397260274, 'right_eye': -69.73684210526316}
{'left_eye': -70.66666666666667, 'right_eye': -69.92084432717678}
{'left_eye': -69.19060052219321, 'right_eye': -67.77493606138107}
{'left_eye': -69.55380577427822, 'right_eye': -67.94871794871794}
{'left_eye': -69

In [ ]:

img_l = cv.imread(f'{SCREENSHOT_PATH["rgb_l"]}101.jpg')
img_r = cv.imread(f'{SCREENSHOT_PATH["rgb_r"]}101.jpg')
# r rgb ir
# img_l = resizeRGB(img_l)
# img_r = resizeIR(img_r)

Left_nice= cv.remap(img_l,Left_Stereo_Map_x,Left_Stereo_Map_y, cv.INTER_LINEAR, cv.BORDER_CONSTANT, 0)
Right_nice= cv.remap(img_r,Right_Stereo_Map_x,Right_Stereo_Map_y, cv.INTER_LINEAR, cv.BORDER_CONSTANT, 0)

# cv.imshow('a',img_l)
# cv.imshow('b',img_r)
# cv.waitKey(0)
# cv.destroyAllWindows()
stereo = 0
if 0:
    minDisparity = 0;
    numDisparities = 480;
    blockSize = 15;
    disp12MaxDiff = 1;
    uniquenessRatio = 1;
    speckleWindowSize = 0;
    speckleRange = 0;

    # Creating an object of StereoSGBM algorithm
    stereo = cv.StereoSGBM_create(minDisparity = minDisparity,
        numDisparities = numDisparities,
        blockSize = blockSize,
        disp12MaxDiff = disp12MaxDiff,
        uniquenessRatio = uniquenessRatio,
        speckleWindowSize = speckleWindowSize,
        speckleRange = speckleRange
    )
else:
    stereo = cv.StereoBM_create()
    stereo.setNumDisparities(160)
    stereo.setBlockSize(11)
#     stereo.setPreFilterType(1)
#     stereo.setPreFilterSize(7)
#     stereo.setPreFilterCap(31)
#     stereo.setTextureThreshold(10)
#     stereo.setUniquenessRatio(15)
#     stereo.setSpeckleRange(0)
#     stereo.setSpeckleWindowSize(0)
#     stereo.setDisp12MaxDiff(0)
#     stereo.setMinDisparity(0)
    
img_gray_l = cv.cvtColor(Left_nice, cv.COLOR_BGR2GRAY)
img_gray_r = cv.cvtColor(Right_nice, cv.COLOR_BGR2GRAY)
# img_gray_l = cv.pyrDown(img_gray_l)
# img_gray_r = cv.pyrDown(img_gray_r)
# img_gray_l = cv.pyrDown(img_gray_l)
# img_gray_r = cv.pyrDown(img_gray_r)
disparity = stereo.compute(img_gray_l,img_gray_r)
Left_nice = cv.cvtColor(markEye(Left_nice), cv.COLOR_BGR2RGB)
Right_nice = cv.cvtColor(markEye(Right_nice), cv.COLOR_BGR2RGB)
img_l = cv.cvtColor(markEyeAndRemappedEye(img_l, Inv_Left_Stereo_Map_x, Inv_Left_Stereo_Map_y), cv.COLOR_BGR2RGB)
img_r = cv.cvtColor(markEyeAndRemappedEye(img_r, Inv_Right_Stereo_Map_x, Inv_Right_Stereo_Map_y), cv.COLOR_BGR2RGB)
# %matplotlib notebook
f, axarr = plt.subplots(3,2)
f.set_figwidth(12)
f.set_figheight(12)
axarr[0,0].imshow(disparity, 'gray')
axarr[0,1].imshow(disparity, 'gray')
axarr[1,0].imshow(Left_nice)
axarr[1,1].imshow(Right_nice)
axarr[2,0].imshow(img_l)
axarr[2,1].imshow(img_r)
plt.show()